In [1]:
#spark_session
from pyspark.sql import SparkSession
spark_session =  SparkSession.builder\
    .master('local[*]')\
    .config("spark.driver.memory", "16g")\
    .config("spark.executor.memory", "16g") \
    .config("spark.memory.fraction", "0.8") \
    .appName('AML_project')\
    .getOrCreate()

In [2]:
df = spark_session.read.csv("../Data/SAML-D.csv",header=True)
df.show()

+--------+----------+--------------+----------------+--------+----------------+-----------------+--------------------+----------------------+------------+-------------+--------------------+
|    Time|      Date|Sender_account|Receiver_account|  Amount|Payment_currency|Received_currency|Sender_bank_location|Receiver_bank_location|Payment_type|Is_laundering|     Laundering_type|
+--------+----------+--------------+----------------+--------+----------------+-----------------+--------------------+----------------------+------------+-------------+--------------------+
|10:35:19|2022-10-07|    8724731955|      2769355426| 1459.15|       UK pounds|        UK pounds|                  UK|                    UK|Cash Deposit|            0|Normal_Cash_Deposits|
|10:35:20|2022-10-07|    1491989064|      8401255335| 6019.64|       UK pounds|           Dirham|                  UK|                   UAE|Cross-border|            0|      Normal_Fan_Out|
|10:35:20|2022-10-07|     287305149|      44047670

In [3]:
from pyspark.sql.functions import year, month, dayofmonth, to_date

df = df.withColumn("Date", to_date(df["Date"], "yyyy-MM-dd"))

df = df.withColumn("year", year(df["Date"])) \
       .withColumn("month", month(df["Date"])) \
       .withColumn("day", dayofmonth(df["Date"]))


In [4]:
from pyspark.sql.functions import to_timestamp,hour,minute,second 

df = df.withColumn("Time", to_timestamp(df["Time"], "HH:mm:ss"))

df = df.withColumn("hour", hour(df["Time"])) \
       .withColumn("minute", minute(df["Time"])) \
       .withColumn("second", second(df["Time"]))


In [5]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
print(f"Training Set: {train_df.count()} rows")
print(f"Testing Set: {test_df.count()} rows")

Training Set: 7603866 rows
Testing Set: 1900986 rows


In [ ]:
from pyspark.sql.functions import col
numeric_cols = ['Is_laundering']

for col_name in numeric_cols:
    train_df = train_df.withColumn(col_name, col(col_name).cast("double"))
    test_df = test_df.withColumn(col_name, col(col_name).cast("double"))